# Подготовка

In [ ]:
import sys
!{sys.executable} -m pip install catboost mljar-supervised scikit-multilearn 

In [ ]:
# !pip install -q -U git+https://github.com/mljar/mljar-supervised.git@master

In [ ]:
# !pip install lightautoml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Olymps/tochka

/content/drive/MyDrive/Olymps/tochka


In [ ]:
# !unzip -q AutoMLs.zip

In [ ]:
import random
import numpy as np
import pandas as pd
import catboost as cb
import sklearn.utils as sku
from skmultilearn.model_selection import iterative_train_test_split
import os
import json
from sklearn.metrics import fbeta_score, classification_report

In [ ]:
from supervised.automl import AutoML

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
# from lightautoml.tasks import Task
# from lightautoml.report.report_deco import ReportDeco

In [ ]:
SEED = 56

random.seed(SEED)
np.random.seed(SEED)

In [ ]:
PAYMENTS_TRAIN_PATH = 'data/payments_train.csv'
TARGET_TRAIN_PATH = 'data/target_train.csv'
PAYMENTS_TEST_PATH = 'data/payments_test.csv'
CLIENT_ID_TEST_PATH = 'data/client_id_test.csv'

# Загружаем датасет

In [ ]:
payments_dtypes = {
    'client_id': str,
    'contractor_id': str,
    'is_outgoing': bool,
    'amount': 'uint64',
    'dt_day': 'uint16',
    'dt_hour': 'uint8',
    'channel': pd.CategoricalDtype()
}
for i in range(12):
    payments_dtypes[f'flag_{i}'] = bool
payments_dtypes

{'amount': 'uint64',
 'channel': CategoricalDtype(categories=None, ordered=False),
 'client_id': str,
 'contractor_id': str,
 'dt_day': 'uint16',
 'dt_hour': 'uint8',
 'flag_0': bool,
 'flag_1': bool,
 'flag_10': bool,
 'flag_11': bool,
 'flag_2': bool,
 'flag_3': bool,
 'flag_4': bool,
 'flag_5': bool,
 'flag_6': bool,
 'flag_7': bool,
 'flag_8': bool,
 'flag_9': bool,
 'is_outgoing': bool}

In [ ]:
payments = pd.read_csv(PAYMENTS_TRAIN_PATH, dtype=payments_dtypes)
payments

,client_id,contractor_id,is_outgoing,amount,dt_day,dt_hour,channel,flag_0,flag_1,flag_2,flag_3,flag_4,flag_5,flag_6,flag_7,flag_8,flag_9,flag_10,flag_11
0,569703,NaN,True,8674442,56,12,app,False,False,False,False,False,False,False,False,False,False,False,False
1,696595,3920,True,5714350,311,19,web,False,False,False,False,False,False,False,False,False,False,False,False
2,368467,NaN,True,3720501,175,13,NaN,False,False,False,True,False,False,False,False,False,False,False,False
3,421133,NaN,True,311542,68,14,NaN,False,False,False,True,False,False,False,False,False,False,False,False
4,365044,24686,True,705918747,171,15,app,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25937985,532940,NaN,True,904019,122,1,pos,True,False,False,False,False,False,False,False,False,False,False,False
25937986,923040,NaN,False,27261705,94,12,NaN,False,False,False,False,False,False,False,False,False,False,False,True
25937987,807703,58960,False,18409310,119,13,NaN,False,False,False,False,False,False,False,False,False,False,False,True
25937988,890561,NaN,True,7132,213,8,NaN,False,False,False,True,False,False,False,False,False,False,False,False


In [ ]:
target_dtypes = {
    'client_id': str
}
for i in range(35):
    target_dtypes[f'type_{i}'] = int
target_dtypes

{'client_id': str,
 'type_0': int,
 'type_1': int,
 'type_10': int,
 'type_11': int,
 'type_12': int,
 'type_13': int,
 'type_14': int,
 'type_15': int,
 'type_16': int,
 'type_17': int,
 'type_18': int,
 'type_19': int,
 'type_2': int,
 'type_20': int,
 'type_21': int,
 'type_22': int,
 'type_23': int,
 'type_24': int,
 'type_25': int,
 'type_26': int,
 'type_27': int,
 'type_28': int,
 'type_29': int,
 'type_3': int,
 'type_30': int,
 'type_31': int,
 'type_32': int,
 'type_33': int,
 'type_34': int,
 'type_4': int,
 'type_5': int,
 'type_6': int,
 'type_7': int,
 'type_8': int,
 'type_9': int}

In [ ]:
target = pd.read_csv(TARGET_TRAIN_PATH, dtype=target_dtypes).set_index('client_id')
target

,type_0,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,...,type_25,type_26,type_27,type_28,type_29,type_30,type_31,type_32,type_33,type_34
client_id,,,,,,,,,,,,,,,,,,,,,
775943,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
992314,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
255821,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0
188791,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
46092,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
406742,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,0,0,0
242611,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Feature Engineering

In [ ]:
def generate_features(pay):
    pay = pay.join(pd.get_dummies(pay['channel']))
    pay = pay.join(pd.get_dummies(pay['is_outgoing']))
    pay.rename(columns={True: 'out', False: 'in'}, inplace=True)
    print(pay)

    fts = pay.groupby('client_id')['amount'].agg(['mean', 'median', 'std', 'min', 'max'])
    fts = fts.join(pay.groupby('client_id')['dt_day'].agg(['std']), 
               how='left', lsuffix='_left', rsuffix='_right')
    fts = fts.join(pay.groupby('client_id')['dt_hour'].agg(['std']), 
               how='left', lsuffix='_left', rsuffix='_right')
    fts[[f'flag_{i}_count' for i in range(12)]] = pay.groupby('client_id')[[f'flag_{i}' for i in range(12)]].sum()
    fts[['app', 'atm', 'pos', 'web']] = pay.groupby('client_id')[['app', 'atm', 'pos', 'web']].sum()
    fts[['in', 'out']] = pay.groupby('client_id')[['in', 'out']].sum()
    fts['sub'] = np.subtract(fts['max'], fts['min'], dtype=np.float32)
    fts['mm'] = np.subtract(fts['mean'], fts['median'], dtype=np.float32)
    fts['total'] = fts[['app', 'atm', 'pos', 'web']].sum(axis=1)
    fts['total_flags'] = fts[[f'flag_{i}_count' for i in range(12)]].sum(axis=1)

    return fts

In [ ]:
features = generate_features(payments)
features

         client_id contractor_id  is_outgoing     amount  dt_day  dt_hour  \
0           569703           NaN         True    8674442      56       12   
1           696595          3920         True    5714350     311       19   
2           368467           NaN         True    3720501     175       13   
3           421133           NaN         True     311542      68       14   
4           365044         24686         True  705918747     171       15   
...            ...           ...          ...        ...     ...      ...   
25937985    532940           NaN         True     904019     122        1   
25937986    923040           NaN        False   27261705      94       12   
25937987    807703         58960        False   18409310     119       13   
25937988    890561           NaN         True       7132     213        8   
25937989    362541           NaN        False    8627951     185        2   

         channel  flag_0  flag_1  flag_2  ...  flag_8  flag_9  flag_10  \
0

,mean,median,std_left,min,max,std_right,std,flag_0_count,flag_1_count,flag_2_count,...,app,atm,pos,web,in,out,sub,mm,total,total_flags
client_id,,,,,,,,,,,,,,,,,,,,,
100045,8.362072e+06,1706470.0,1.733687e+07,22,124737631,119.830912,6.193097,203,0,60,...,1.0,54.0,121.0,17.0,101.0,376.0,124737609,6.655602e+06,193.0,320
100055,2.201262e+07,4010524.0,6.029931e+07,2238,766121312,114.166663,5.061865,206,0,42,...,0.0,79.0,99.0,158.0,136.0,515.0,766119074,1.800210e+07,336.0,538
100068,3.782283e+07,3232108.0,8.493012e+07,34043,681967564,44.190426,5.491089,394,0,22,...,37.0,22.0,320.0,14.0,62.0,457.0,681933521,3.459072e+07,393.0,461
100076,1.155591e+07,1522173.0,3.589745e+07,29,255864840,42.746639,6.759718,63,0,11,...,6.0,10.0,26.0,7.0,9.0,84.0,255864811,1.003374e+07,49.0,102
100089,3.065695e+07,21997923.0,4.485693e+07,1519,645257028,107.477730,7.547993,264,0,52,...,243.0,62.0,92.0,21.0,925.0,659.0,645255509,8.659025e+06,418.0,1362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99975,5.391718e+07,1333079.0,1.590713e+08,4660,859557956,43.546906,5.764104,20,0,1,...,1.0,2.0,0.0,0.0,5.0,45.0,859553296,5.258410e+07,3.0,50
999759,2.389214e+08,1465890.0,1.122544e+09,4742,6612336710,110.050365,6.584230,158,0,0,...,0.0,0.0,135.0,12.0,46.0,227.0,6612331968,2.374555e+08,147.0,178
999784,3.565689e+08,18552982.0,7.886591e+08,28594,3594312408,81.121899,5.430926,0,0,0,...,2.0,0.0,0.0,41.0,16.0,69.0,3594283814,3.380159e+08,43.0,56


## Разбиение датасета на train / val

Реализуем функцию для разбиения датасета на обучающую и валидационную выборки в пропорциях 85%/15% соответвенно, примерно сохраняя распределение таргетов в обеих выборках. Из соображений быстродействия и детерминированности будем кешировать разбиение в JSON-файл. 

Для стратификации используется функция [iterative_train_test_split](http://scikit.ml/_modules/skmultilearn/model_selection/iterative_stratification.html#iterative_train_test_split) из библиотечки scikit-multilearn.

In [ ]:
def stratified_split_cached(X, y, split_idx_file):
    if os.path.isfile(split_idx_file):
        with open(split_idx_file, 'r') as f:
            split_json = json.load(f)
        train_idx, val_idx = split_json['train'], split_json['val']
    else:
        y_shuffle = sku.shuffle(y, random_state=SEED)  # https://cpb-us-e1.wpmucdn.com/journeys.dartmouth.edu/dist/8/830/files/2020/06/EIqwWwsX0AAeh-o.jpeg
        train_idx, _, val_idx, _ = iterative_train_test_split(np.expand_dims(y_shuffle.index, 1), np.array(y_shuffle), test_size=0.05)
        train_idx, val_idx = train_idx.squeeze(1), val_idx.squeeze(1)
        with open(split_idx_file, 'w') as f:
            json.dump({'train': list(train_idx), 'val': list(val_idx)}, f)
    return X.loc[train_idx], y.loc[train_idx], X.loc[val_idx], y.loc[val_idx]

In [ ]:
cache = 'split_cache_3.json'
X_train, y_train, X_val, y_val = stratified_split_cached(features, target, cache)

In [ ]:
X_train.shape

(60378, 29)

In [ ]:
temp = X_train
temp = temp.join(y_train)
temp

,mean,median,std_left,min,max,std_right,std,flag_0_count,flag_1_count,flag_2_count,...,type_25,type_26,type_27,type_28,type_29,type_30,type_31,type_32,type_33,type_34
client_id,,,,,,,,,,,,,,,,,,,,,
469347,3.544950e+07,13163196.5,7.754265e+07,12380,1098312208,108.508350,5.742443,57,0,35,...,0,0,0,0,0,1,0,0,0,0
438388,2.085516e+07,2544355.5,3.834703e+07,2409,170421023,87.565945,6.509380,97,0,21,...,0,0,1,0,0,1,0,0,0,1
190365,1.506990e+07,2655680.0,5.225610e+07,37331,671071999,64.844801,6.554789,334,0,10,...,1,1,0,0,0,0,1,0,0,0
410390,2.424419e+07,11531806.0,4.460235e+07,100115,220204345,103.452385,6.367848,0,0,0,...,0,0,0,1,0,1,0,0,1,0
517381,5.120096e+07,5231211.5,7.478513e+07,153,199978074,91.157072,6.152720,4,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399347,3.668457e+07,4704610.0,6.428507e+07,63384,357004808,125.030299,4.240102,22,0,21,...,0,0,0,0,0,1,0,0,1,0
470557,4.295133e+07,1210504.0,8.892837e+07,2928,506319917,58.239720,5.146303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
42811,5.099055e+07,3634531.0,1.492271e+08,88,991314308,94.376220,6.870176,151,41,17,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# col = 'type_0'
feat = 'total_flags'
imps = []
for i in range(35):
    imps.append(temp.corr().sort_values([f'type_{i}'], ascending=False)[f'type_{i}'][feat])
print(sum(imps) / 35, min(imps), max(imps)) 

-0.008657623372820097 -0.04426022099160501 0.11071837757495642


In [ ]:
col = 'type_22'
temp.corr().sort_values([col], ascending=False)[col].iloc[:15]

type_22         1.000000
type_11         0.499614
type_34         0.214233
type_24         0.209601
type_12         0.183180
type_0          0.154214
type_5          0.116024
type_13         0.093855
flag_3_count    0.084315
type_18         0.074300
atm             0.074244
app             0.070226
total           0.065331
flag_2_count    0.062142
out             0.060654
Name: type_22, dtype: float64

# Catboost 3x Ensemble

In [ ]:
def make_pool(X, y=None):
    return cb.Pool(X, y)

In [ ]:
models = []
models_inv = []
clfs = []

X_train, y_train, X_val, y_val = stratified_split_cached(features, target, cache)

for i in range(35):
    print('\nFitting model', i)
    # print(X_train.columns)
    if i == 0 or i == 30:
        lr = 0.15
    else:
        lr = 0.45
    model = cb.CatBoostClassifier(loss_function='Logloss', random_seed=SEED, learning_rate=lr, 
                                  l2_leaf_reg=500, bagging_temperature=100) # iterations=400, learning_rate=0.1, bagging_temperature=2 l2_leaf_reg=1, learning_rate=0.1)
    smote = SMOTE()
    X_sm, y_sm = smote.fit_resample(X_train, y_train[f'type_{i}'])
    print('Data augmented')
    print(y_sm.value_counts())
    pool_train, pool_val = make_pool(X_sm, y_sm), make_pool(X_val, y_val[f'type_{i}'])
    model.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
    models.append(model)

    model_inv = cb.CatBoostClassifier(loss_function='Logloss', random_seed=SEED, learning_rate=lr, # iterations=200, learning_rate=0.1, 
                                      l2_leaf_reg=200, bagging_temperature=20)
    idk = pd.DataFrame()
    idk_val = pd.DataFrame()
    idk[f'type_{i}_inv'] = y_sm.replace({0: 1, 1: 0})
    idk_val[f'type_{i}_inv'] = y_val[f'type_{i}'].replace({0: 1, 1: 0})
    pool_train, pool_val = make_pool(X_sm, idk[f'type_{i}_inv']), make_pool(X_val, idk_val[f'type_{i}_inv'])
    model_inv.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
    models_inv.append(model_inv)

    preds = np.array(y_train[f'type_{i}']) #.transpose(1, 0)
    print(np.array(model.predict_proba(X_train)).shape)
    probs = pd.DataFrame(np.array(model.predict_proba(X_train))) #.transpose(1, 2, 0).reshape(X_train.shape[0], 70))
    probs_inv = pd.DataFrame(np.array(model_inv.predict_proba(X_train))).rename(columns={0: 2, 1: 3}) #.transpose(1, 2, 0).reshape(X_train.shape[0], 70))
    df = pd.concat([probs, probs_inv], axis=1)
    thresh = int(len(preds) * 0.85)
    clf = cb.CatBoostClassifier(iterations=300, loss_function='Logloss', random_seed=SEED, learning_rate=0.1, 
                                l2_leaf_reg=100, bagging_temperature=10)
    pool_train, pool_val = make_pool(df[:thresh], preds[:thresh]), make_pool(df[thresh:], preds[thresh:]) 
    clf.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
    clfs.append(clf)

    X_train[f'type_{i}'] = y_train[f'type_{i}'] # model.predict(X_train)
    # X_train[f'type_{i}_inv'] = y_train[f'type_{i}_inv']
    probs = pd.DataFrame(np.array(model.predict_proba(X_val))) #.transpose(1, 2, 0).reshape(X_train.shape[0], 70))
    probs_inv = pd.DataFrame(np.array(model_inv.predict_proba(X_val))).rename(columns={0: 2, 1: 3})
    df = pd.concat([probs, probs_inv], axis=1)
    X_val[f'type_{i}'] = clf.predict(df)
    # X_val[f'type_{i}_inv'] = model_inv.predict(X_val)
    print('fbeta:', fbeta_score(y_val[f'type_{i}'], X_val[f'type_{i}'], beta=0.5, average='micro', zero_division=0))
    print(X_val[f'type_{i}'].value_counts())
    print(y_val[f'type_{i}'].value_counts())

    # break
  


Fitting model 0
Data augmented
1    39896
0    39896
Name: type_0, dtype: int64
0:	learn: 0.6757652	test: 0.6796539	best: 0.6796539 (0)	total: 33.9ms	remaining: 33.9s
100:	learn: 0.5560469	test: 0.6137052	best: 0.6134515 (98)	total: 3.22s	remaining: 28.7s
200:	learn: 0.5338798	test: 0.6085726	best: 0.6084084 (186)	total: 6.23s	remaining: 24.8s
300:	learn: 0.5155405	test: 0.6046349	best: 0.6045496 (298)	total: 9.29s	remaining: 21.6s
400:	learn: 0.5052104	test: 0.6031332	best: 0.6031332 (400)	total: 12.4s	remaining: 18.5s
500:	learn: 0.4982281	test: 0.6030522	best: 0.6027751 (465)	total: 15.4s	remaining: 15.3s
600:	learn: 0.4919953	test: 0.6025584	best: 0.6024738 (589)	total: 18.4s	remaining: 12.2s
700:	learn: 0.4848874	test: 0.6014755	best: 0.6013547 (692)	total: 21.5s	remaining: 9.16s
800:	learn: 0.4789292	test: 0.6021543	best: 0.6013547 (692)	total: 24.6s	remaining: 6.12s
900:	learn: 0.4733572	test: 0.6024748	best: 0.6013547 (692)	total: 27.6s	remaining: 3.04s
999:	learn: 0.4686314	t

# skl ensemble

In [ ]:
from sklearn.ensemble import RandomForestClassifier

models = []
knns = []
X_train, y_train, X_val, y_val = stratified_split_cached(features, target, 'split_cache.json')
print(X_train.shape)

for i in range(35):
    print('Fitting model', i)
    # print(X_train.columns)
    # model = cb.CatBoostRegressor(iterations=300, loss_function='RMSE', random_seed=SEED) #, learning_rate=0.1, 
                                  # l2_leaf_reg=100, bagging_temperature=10) # iterations=300, learning_rate=0.1, bagging_temperature=2 l2_leaf_reg=1, learning_rate=0.1)
    # pool_train, pool_val = make_pool(X_train, y_train[f'type_{i}']), make_pool(X_val, y_val[f'type_{i}'])
    # model.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
    model = RandomForestClassifier()
    print(X_train.columns, len(X_train.columns))
    model.fit(X_train, y_train[f'type_{i}'])
    knns.append(model)
    X_train[f'type_{i}'] = y_train[f'type_{i}'] # model.predict(X_train)
    # print(model.predict(X_val))
    # X_val[f'type_{i}'] = model.predict(X_val)
    

(60378, 29)
Fitting model 0
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_count', 'flag_7_count',
       'flag_8_count', 'flag_9_count', 'flag_10_count', 'flag_11_count', 'app',
       'atm', 'pos', 'web', 'in', 'out', 'sub', 'mm', 'total', 'total_flags'],
      dtype='object') 29
Fitting model 1
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_count', 'flag_7_count',
       'flag_8_count', 'flag_9_count', 'flag_10_count', 'flag_11_count', 'app',
       'atm', 'pos', 'web', 'in', 'out', 'sub', 'mm', 'total', 'total_flags',
       'type_0'],
      dtype='object') 30
Fitting model 2
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'fl

In [ ]:
len(X_val.columns)

29

# MLJar

In [ ]:
automls = []

X_train, y_train, X_val, y_val = stratified_split_cached(features, target, 'split_cache.json')

for i in range(35):
    automl = AutoML(results_path=f"AutoMLs/AutoML_{i + 1}")
    X_train[f'type_{i}'] = y_train[f'type_{i}']
    X_val[f'type_{i}'] = automl.predict(X_val)
    print(i)
    automls.append(automl)

# for i in range(50, 62):
#     print(0)
#     automl = AutoML(results_path=f"AutoML_{i}")
#     X_train[f'type_{i - 34}'] = y_train[f'type_{i}']
#     X_val[f'type_{i - 34}'] = automl.predict(X_val)
#     print(i)
#     automls.append(automl)

In [ ]:
X_val.columns

Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_count', 'flag_7_count',
       'flag_8_count', 'flag_9_count', 'flag_10_count', 'flag_11_count', 'app',
       'atm', 'pos', 'web', 'in', 'out', 'sub', 'mm', 'total', 'total_flags',
       'type_0', 'type_1', 'type_2', 'type_3', 'type_4', 'type_5', 'type_6',
       'type_7', 'type_8', 'type_9', 'type_10', 'type_11', 'type_12',
       'type_13', 'type_14', 'type_15', 'type_16', 'type_17', 'type_18',
       'type_19', 'type_20', 'type_21', 'type_22', 'type_23', 'type_24',
       'type_25', 'type_26', 'type_27', 'type_28', 'type_29', 'type_30',
       'type_31', 'type_32', 'type_33', 'type_34'],
      dtype='object')

In [ ]:
X_train.shape, X_val.shape

((60378, 53), (6627, 53))

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X_train, y_train['type_0'])
X_sm

,mean,median,std_left,min,max,std_right,std,flag_0_count,flag_1_count,flag_2_count,...,app,atm,pos,web,in,out,sub,mm,total,total_flags
0,3.544950e+07,1.316320e+07,7.754265e+07,12380,1098312208,108.508350,5.742443,57,0,35,...,122.000000,33.000000,7.000000,6.000000,253.000000,265.000000,1098299828,2.228631e+07,168.000000,461
1,2.085516e+07,2.544356e+06,3.834703e+07,2409,170421023,87.565945,6.509380,97,0,21,...,28.000000,21.000000,59.000000,28.000000,23.000000,161.000000,170418614,1.831081e+07,136.000000,179
2,1.506990e+07,2.655680e+06,5.225610e+07,37331,671071999,64.844801,6.554789,334,0,10,...,6.000000,12.000000,168.000000,0.000000,16.000000,376.000000,671034668,1.241422e+07,186.000000,391
3,2.424419e+07,1.153181e+07,4.460235e+07,100115,220204345,103.452385,6.367848,0,0,0,...,10.000000,0.000000,0.000000,13.000000,17.000000,30.000000,220104230,1.271239e+07,23.000000,11
4,5.120096e+07,5.231212e+06,7.478513e+07,153,199978074,91.157072,6.152720,4,0,0,...,2.000000,0.000000,0.000000,20.000000,22.000000,54.000000,199977921,4.596975e+07,22.000000,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75671,1.812006e+07,1.806865e+06,9.467230e+07,13045,1505140311,70.735143,5.050416,489,0,23,...,5.282923,37.770624,397.541959,1.848770,10.000000,538.751798,1505127265,1.631319e+07,442.444277,567
75672,3.883716e+07,1.382910e+06,1.344486e+08,10533,980450810,50.725831,4.485564,424,0,24,...,13.860242,25.334780,385.031400,18.383535,24.622973,466.908997,980440276,3.745425e+07,442.609957,474
75673,8.620434e+07,9.215106e+06,1.427771e+08,30008,936122723,80.620121,5.312969,64,0,14,...,0.000000,16.361771,47.489975,49.638229,31.085314,177.724008,936092714,7.698924e+07,113.489975,165
75674,2.237747e+07,1.065435e+06,3.669905e+07,20023,105804073,89.774830,5.631048,63,0,22,...,0.112390,26.100879,35.426538,0.112390,17.719025,87.358832,105784049,2.131203e+07,61.752197,112


In [ ]:
y_sm.value_counts()

0    37838
1    37838
Name: type_0, dtype: int64

In [ ]:
len(automls)

24

In [ ]:
# automls = []

# X_train, y_train, X_val, y_val = stratified_split_cached(features, target, 'split_cache.json')
X_val_temp = X_val

for i in range(24, 35):
    print('Fitting model', i)
    # print(X_train.columns)
    # model = cb.CatBoostRegressor(iterations=300, loss_function='RMSE', random_seed=SEED) #, learning_rate=0.1, 
                                  # l2_leaf_reg=100, bagging_temperature=10) # iterations=300, learning_rate=0.1, bagging_temperature=2 l2_leaf_reg=1, learning_rate=0.1)
    # pool_train, pool_val = make_pool(X_train, y_train[f'type_{i}']), make_pool(X_val, y_val[f'type_{i}'])
    # model.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)

    automl = AutoML(mode='Compete', algorithms=['Decision Tree', 'Extra Trees', 'LightGBM', 'CatBoost'],
                    start_random_models=5, total_time_limit=240, eval_metric='logloss')
    automl.fit(X_train, y_train[f'type_{i}'])
    preds = automl.predict(X_val)
    print('fbeta:', fbeta_score(y_val[f'type_{i}'], preds, beta=0.5, average='micro', zero_division=0))
    X_train[f'type_{i}'] = y_train[f'type_{i}'] # model.predict(X_train)
    X_val[f'type_{i}'] = automl.predict(X_val)
    print(X_val[f'type_{i}'].value_counts())
    # print(model.predict(X_val))
    # X_val[f'type_{i}'] = model.predict(X_val)
    automls.append(automl)


Fitting model 24
AutoML directory: AutoML_14
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Extra Trees', 'LightGBM', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree logloss 0.327355 trained in 2.66 seconds
Disable stacking for split validation
* Step simple_algorithms will try to check up to 2 models
2_DecisionTree logloss 0.32652 trained in 2.7 seconds
3_DecisionTree logloss 0.32652 trained in 2.62 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_LightGBM logloss 0.321895 trained in 5.33 seconds
5_Default_CatBoost logloss

In [ ]:
automls

[AutoML(results_path='AutoMLs/AutoML_1'),
 AutoML(results_path='AutoMLs/AutoML_2'),
 AutoML(results_path='AutoMLs/AutoML_3'),
 AutoML(results_path='AutoMLs/AutoML_4'),
 AutoML(results_path='AutoMLs/AutoML_5'),
 AutoML(results_path='AutoMLs/AutoML_6'),
 AutoML(results_path='AutoMLs/AutoML_7'),
 AutoML(results_path='AutoMLs/AutoML_8'),
 AutoML(results_path='AutoMLs/AutoML_9'),
 AutoML(results_path='AutoMLs/AutoML_10'),
 AutoML(results_path='AutoMLs/AutoML_11'),
 AutoML(results_path='AutoMLs/AutoML_12'),
 AutoML(results_path='AutoMLs/AutoML_13'),
 AutoML(results_path='AutoMLs/AutoML_14'),
 AutoML(results_path='AutoMLs/AutoML_15'),
 AutoML(results_path='AutoMLs/AutoML_16'),
 AutoML(results_path='AutoMLs/AutoML_17'),
 AutoML(results_path='AutoMLs/AutoML_18'),
 AutoML(results_path='AutoMLs/AutoML_19'),
 AutoML(results_path='AutoMLs/AutoML_20'),
 AutoML(results_path='AutoMLs/AutoML_21'),
 AutoML(results_path='AutoMLs/AutoML_22'),
 AutoML(results_path='AutoMLs/AutoML_23'),
 AutoML(results_path

Saved: AutoML_16 - AutoML_28

In [ ]:
['Decision Tree', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Nearest Neighbors']

# LightAutoML

In [ ]:
lmls = []

X_train, y_train, X_val, y_val = stratified_split_cached(features, target, 'split_cache.json')

for i in range(35):
    print('Fitting model', i)
    # print(X_train.columns)
    # model = cb.CatBoostRegressor(iterations=300, loss_function='RMSE', random_seed=SEED) #, learning_rate=0.1, 
                                  # l2_leaf_reg=100, bagging_temperature=10) # iterations=300, learning_rate=0.1, bagging_temperature=2 l2_leaf_reg=1, learning_rate=0.1)
    # pool_train, pool_val = make_pool(X_train, y_train[f'type_{i}']), make_pool(X_val, y_val[f'type_{i}'])
    # model.fit(pool_train, eval_set=pool_val, plot=False, verbose=100)
    task = Task('binary')
    N_THREADS = 4
    N_FOLDS = 5
    RANDOM_STATE = 25
    TEST_SIZE = 0.2
    TIMEOUT = 300
    TARGET_NAME = f'type_{i}'
    roles = {
        'target': TARGET_NAME,
        # 'drop': ['SK_ID_CURR']
    }
    lml = TabularAutoML(
        task=task,
        timeout=TIMEOUT,
        cpu_limit=N_THREADS,
        reader_params={'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
    )
    df = pd.concat([X_train, y_train[f'type_{i}']], axis=1)
    # print(X_train.columns, len(X_train.columns))
    lml.fit(df, roles=roles, verbose=2)
    lmls.append(lml)
    X_train[f'type_{i}'] = y_train[f'type_{i}'] # model.predict(X_train)
    # print(model.predict(X_val))
    # X_val[f'type_{i}'] = model.predict(X_val)
  

# Проверяем качество модели на валидации

Catboost predict function

In [ ]:
def predict(X, models, models_inv, clfs):
    # preds = []
    # probs = []
    # probs_inv = []
    res = []
    i = 0
    for model, model_inv, clf in zip(models, models_inv, clfs): 
        print(i)
        print(X.columns)
        # preds.append(model.predict(X)) # (make_pool(X)))
        # X[f'type_{i}'] = preds[i]
        # X[f'type_{i}_inv'] = model_inv.predict(X)
        # probs.append(model.predict_proba(X))
        # probs_inv.append(model_inv.predict_proba(X))
        # print(X.columns)
        # X_train[f'type_{i}'] = y_train[f'type_{i}'] # model.predict(X_train)
        # X_train[f'type_{i}_inv'] = y_train[f'type_{i}_inv']
        probs = pd.DataFrame(np.array(model.predict_proba(X))) #.transpose(1, 2, 0).reshape(X_train.shape[0], 70))
        probs_inv = pd.DataFrame(np.array(model_inv.predict_proba(X))).rename(columns={0: 2, 1: 3})
        df = pd.concat([probs, probs_inv], axis=1)
        X[f'type_{i}'] = clf.predict(df)
        res.append(X[f'type_{i}'])
        i += 1

    print(res)
    preds = pd.DataFrame(np.array(res).transpose(1, 0), index=X.index, columns=[f'type_{i}' for i in range(35)]).astype(int)
    # print(np.array(probs).shape, np.array(probs).T.shape, np.array(probs).T.T.shape)
    # print(np.array(probs).transpose(1, 2, 0).reshape(6627, 70).shape)
    # probs = pd.DataFrame(np.array(probs).transpose(1, 2, 0).reshape(6627, 70))
    # probs_inv = pd.DataFrame(np.array(probs_inv).transpose(1, 2, 0).reshape(6627, 70))
    # print(0)
    return preds #, probs, probs_inv

MLJar prediction func

In [ ]:
def predict(X, clfs):
    res = []
    i = 0
    idx = X.index
    for clf in clfs: 
        print(i)
        # print(X.columns)
        res.append(clf.predict(X)) # (make_pool(X)))
        X[f'type_{i}'] = res[i]
        # X[f'type_{i}_inv'] = model_inv.predict(X)
        # probs.append(model.predict_proba(X))
        # probs_inv.append(model_inv.predict_proba(X))
        # print(X.columns)
        # X_train[f'type_{i}'] = y_train[f'type_{i}'] # model.predict(X_train)
        # X_train[f'type_{i}_inv'] = y_train[f'type_{i}_inv']
        # probs = pd.DataFrame(np.array(model.predict_proba(X))) #.transpose(1, 2, 0).reshape(X_train.shape[0], 70))
        # probs_inv = pd.DataFrame(np.array(model_inv.predict_proba(X))).rename(columns={0: 2, 1: 3})
        # df = pd.concat([probs, probs_inv], axis=1)
        # X[f'type_{i}'] = clf.predict(df)
        # res.append(X[f'type_{i}'])
        i += 1

    print(res)
    print(X.index)
    preds = pd.DataFrame(np.array(res).transpose(1, 0), index=idx, columns=[f'type_{i}' for i in range(35)]).astype(int)
    # print(np.array(probs).shape, np.array(probs).T.shape, np.array(probs).T.T.shape)
    # print(np.array(probs).transpose(1, 2, 0).reshape(6627, 70).shape)
    # probs = pd.DataFrame(np.array(probs).transpose(1, 2, 0).reshape(6627, 70))
    # probs_inv = pd.DataFrame(np.array(probs_inv).transpose(1, 2, 0).reshape(6627, 70))
    # print(0)
    return preds #, probs, probs_inv

In [ ]:
# print(knns)
X_train, y_train, X_val, y_val = stratified_split_cached(features, target, cache)
preds = predict(X_val, models, models_inv, clfs)
preds

0
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_count', 'flag_7_count',
       'flag_8_count', 'flag_9_count', 'flag_10_count', 'flag_11_count', 'app',
       'atm', 'pos', 'web', 'in', 'out', 'sub', 'mm', 'total', 'total_flags'],
      dtype='object')
1
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_count', 'flag_7_count',
       'flag_8_count', 'flag_9_count', 'flag_10_count', 'flag_11_count', 'app',
       'atm', 'pos', 'web', 'in', 'out', 'sub', 'mm', 'total', 'total_flags',
       'type_0'],
      dtype='object')
2
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_

,type_0,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,...,type_25,type_26,type_27,type_28,type_29,type_30,type_31,type_32,type_33,type_34
client_id,,,,,,,,,,,,,,,,,,,,,
467692,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
725233,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
411940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
48344,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
691414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877775,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
870586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
716050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_val

,mean,median,std_left,min,max,std_right,std,flag_0_count,flag_1_count,flag_2_count,...,type_25,type_26,type_27,type_28,type_29,type_30,type_31,type_32,type_33,type_34
0,8.018193e+07,35150673.0,1.162911e+08,73,594908243,111.414450,6.299533,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1.782003e+08,166450463.5,1.873942e+08,31134,726200213,65.966823,4.649342,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5.818194e+07,40526128.0,6.463688e+07,238581,314723245,40.978346,5.508001,15,0,3,...,0,0,0,0,0,0,0,0,0,1
3,1.775031e+07,3203933.5,4.295851e+07,44610,376500259,112.715218,5.991767,43,0,26,...,0,0,0,0,0,0,0,1,0,0
4,1.564197e+07,11660683.5,1.796033e+07,64964,94829059,98.189549,5.420227,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6622,5.649328e+06,4141860.0,5.993685e+06,863,57939883,104.655057,3.329856,26,0,0,...,0,0,0,0,0,0,0,0,0,0
6623,1.001077e+07,488953.5,3.112839e+07,34985,186732089,22.930726,6.818342,102,0,0,...,0,0,0,0,0,0,0,0,0,0
6624,7.886940e+07,36277051.0,1.196606e+08,31694,988117205,118.044056,4.384319,16,0,4,...,0,0,0,0,1,1,0,0,0,0
6625,9.261000e+07,24745941.5,2.212943e+08,30760,1287661678,85.683423,3.596819,43,0,42,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(35):
    print(preds[f'type_{i}'].value_counts())

In [ ]:
print(fbeta_score(y_val, preds, beta=0.5, average='micro', zero_division=0))

0.3385779725154352


# Загружаем тестовые данные

In [ ]:
payments_test = pd.read_csv(PAYMENTS_TEST_PATH, dtype=payments_dtypes)
payments_test

,client_id,contractor_id,is_outgoing,amount,dt_day,dt_hour,channel,flag_0,flag_1,flag_2,flag_3,flag_4,flag_5,flag_6,flag_7,flag_8,flag_9,flag_10,flag_11
0,303546,NaN,True,378449,50,2,NaN,False,False,False,True,False,False,False,False,False,False,False,False
1,59719,NaN,True,281527,267,2,pos,True,False,False,False,False,False,False,False,False,False,False,False
2,25428,90165,False,2105146744,294,9,NaN,False,False,False,False,False,False,False,False,False,False,False,False
3,766314,NaN,True,2156082,207,14,pos,True,False,False,False,False,False,False,False,False,False,False,False
4,465049,NaN,True,51694546,211,3,atm,True,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8553957,794745,773056,False,858062290,198,16,NaN,False,False,False,False,False,False,False,False,False,False,False,False
8553958,362168,904134,False,8540123,130,13,NaN,False,False,False,False,False,False,False,False,False,False,False,False
8553959,500006,471487,False,65346645,196,12,NaN,False,False,False,False,False,False,False,False,False,False,False,False
8553960,653741,NaN,True,3296030,43,0,NaN,True,False,False,False,False,False,False,False,False,False,False,False


# Генерируем фичи для тестовых данных

In [ ]:
features_test = generate_features(payments_test)

        client_id contractor_id  is_outgoing      amount  dt_day  dt_hour  \
0          303546           NaN         True      378449      50        2   
1           59719           NaN         True      281527     267        2   
2           25428         90165        False  2105146744     294        9   
3          766314           NaN         True     2156082     207       14   
4          465049           NaN         True    51694546     211        3   
...           ...           ...          ...         ...     ...      ...   
8553957    794745        773056        False   858062290     198       16   
8553958    362168        904134        False     8540123     130       13   
8553959    500006        471487        False    65346645     196       12   
8553960    653741           NaN         True     3296030      43        0   
8553961    116407           NaN         True      171711     362        3   

        channel  flag_0  flag_1  flag_2  ...  flag_8  flag_9  flag_10  \
0 

In [ ]:
# features_test = features_test.join(pd.get_dummies(features_test['channel']))
features_test

,mean,median,std_left,min,max,std_right,std,flag_0_count,flag_1_count,flag_2_count,...,app,atm,pos,web,in,out,sub,mm,total,total_flags
client_id,,,,,,,,,,,,,,,,,,,,,
100050,3.536920e+07,4800238.5,7.702827e+07,798,474601237,109.455485,5.887787,7,0,0,...,7.0,0.0,0.0,9.0,43.0,117.0,474600439,3.056896e+07,16.0,68
100128,2.074924e+08,42574180.0,5.213882e+08,2401,4263233615,98.462638,4.135433,0,0,0,...,44.0,0.0,0.0,1.0,19.0,78.0,4263231214,1.649182e+08,45.0,65
100159,1.516675e+08,79896060.0,2.081792e+08,17615,1348514255,111.647025,3.939866,0,0,0,...,0.0,0.0,0.0,434.0,298.0,448.0,1348496640,7.177142e+07,434.0,116
10018,5.424148e+07,26540880.5,1.042072e+08,43,808270306,74.408455,3.836922,0,0,0,...,59.0,0.0,0.0,8.0,252.0,98.0,808270263,2.770060e+07,67.0,44
100237,2.005865e+08,50737877.0,3.939886e+08,31511,2521806684,61.305243,5.941169,16,0,12,...,34.0,12.0,4.0,11.0,17.0,89.0,2521775173,1.498486e+08,61.0,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999572,4.462335e+07,21529563.0,8.773594e+07,2137,1586323663,111.300051,3.980125,0,0,0,...,315.0,0.0,0.0,94.0,334.0,436.0,1586321526,2.309379e+07,409.0,181
99966,5.238860e+07,950112.0,2.225828e+08,578,2924402093,114.655475,5.666401,701,0,14,...,69.0,13.0,599.0,22.0,50.0,808.0,2924401515,5.143849e+07,703.0,794
999662,3.545021e+07,5060146.5,9.601183e+07,19606,947093072,91.680202,6.911623,232,0,20,...,0.0,20.0,169.0,84.0,69.0,331.0,947073466,3.039006e+07,273.0,301


# Предскажем значения для тестовых данных и сгенерируем сабмит

In [ ]:
# for i in range(35):
#     features_test[f'type_{i}'] = knns[i].predict(features_test)
# X_train, y_train, X_val, y_val = stratified_split_cached(features, target, 'split_cache.json')
preds_test = predict(features_test, models, models_inv, clfs)
preds_test

0
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_count', 'flag_7_count',
       'flag_8_count', 'flag_9_count', 'flag_10_count', 'flag_11_count', 'app',
       'atm', 'pos', 'web', 'in', 'out', 'sub', 'mm', 'total', 'total_flags'],
      dtype='object')
1
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_count', 'flag_7_count',
       'flag_8_count', 'flag_9_count', 'flag_10_count', 'flag_11_count', 'app',
       'atm', 'pos', 'web', 'in', 'out', 'sub', 'mm', 'total', 'total_flags',
       'type_0'],
      dtype='object')
2
Index(['mean', 'median', 'std_left', 'min', 'max', 'std_right', 'std',
       'flag_0_count', 'flag_1_count', 'flag_2_count', 'flag_3_count',
       'flag_4_count', 'flag_5_count', 'flag_6_

,type_0,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,...,type_25,type_26,type_27,type_28,type_29,type_30,type_31,type_32,type_33,type_34
client_id,,,,,,,,,,,,,,,,,,,,,
100050,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
100128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100159,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10018,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100237,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999572,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
99966,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
999662,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
preds_test.to_csv('submission_aug3.csv')